In [1]:
from BayesianHoldem import BayesianHoldem
import torch

model = BayesianHoldem()

for i in range(10):
    card_representation = torch.randn(6, 13, 4)
    action_representation = torch.randn(24, 4, 4)
    prediction = model.predict_action(action_representation, card_representation)
    if prediction == 0:
        print(f"prediction {i}: Fold")
    elif prediction == 1:
        print(f"prediction {i}: Check/Call")
    elif prediction == 2:
        print(f"prediction {i}: Bet/Raise (BB)")
    elif prediction == 3:
        print(f"prediction {i}: All-in")

prediction 0: Fold
prediction 1: Check/Call
prediction 2: Bet/Raise (BB)
prediction 3: Bet/Raise (BB)
prediction 4: All-in
prediction 5: All-in
prediction 6: Fold
prediction 7: All-in
prediction 8: All-in
prediction 9: Fold


In [4]:
import pokerkit as pk

big_blind = 100
small_blind = 0.5*big_blind
blinds = [small_blind, big_blind]
ante = 0
min_bet = 0.5*big_blind
player_count = 2

P1_stack = 100*big_blind
P2_stack = 50*big_blind

game = pk.NoLimitTexasHoldem(
    # Automations - everything is automatic except for player actions
    (
        pk.Automation.ANTE_POSTING,
        pk.Automation.BET_COLLECTION,
        pk.Automation.BLIND_OR_STRADDLE_POSTING,
        pk.Automation.CARD_BURNING,
        pk.Automation.HOLE_DEALING,
        pk.Automation.BOARD_DEALING,
        pk.Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
        pk.Automation.HAND_KILLING,
        pk.Automation.CHIPS_PUSHING,
        pk.Automation.CHIPS_PULLING,
    ),
    ante_trimming_status=True, # irrelevant but necessary for the game to be created
    raw_antes=ante,
    raw_blinds_or_straddles=blinds,
    min_bet=min_bet,
)

state = game(raw_starting_stacks=[P1_stack, P2_stack], player_count=player_count)
print(type(state))
print(f"Street index: {state.street_index}")
print(f"Stacks: {state.stacks}")

<class 'pokerkit.state.State'>
Street index: 0
Stacks: [9900, 4950.0]


In [5]:
print(type(state.operations[-1]))
state.complete_bet_or_raise_to(500)
state.check_or_call()
print(type(state.operations[-1]))
state.check_or_call()
state.check_or_call()
print(type(state.operations[-1]))
print(tuple(state.pot_amounts)[0])


<class 'pokerkit.state.HoleDealing'>
<class 'pokerkit.state.BoardDealing'>
<class 'pokerkit.state.BoardDealing'>
1000


In [3]:
print(f"Bets: {state.bets}")
print(f"Stacks: {state.stacks}")
print(f"Actor indices: {state.actor_indices}")


state.complete_bet_or_raise_to(state.stacks[state.actor_index] + state.bets[state.actor_index])
print(f"All in")
print(f"Bets: {state.bets}")
print(f"Stacks: {state.stacks}")
print(f"Actor indices: {state.actor_indices}")

state.check_or_call()
print("Check or call")
print(f"Bets: {state.bets}")
print(f"Stacks: {state.stacks}")
print(f"Actor indices: {state.actor_indices}")

print(f"Street index: {state.street_index}")

Bets: [100, 50.0]
Stacks: [9900, 4950.0]
Actor indices: deque([1, 0])
All in
Bets: [100, 5000.0]
Stacks: [9900, 0.0]
Actor indices: deque([0])
Check or call
Bets: [0, 0]
Stacks: [5000.0, 10000.0]
Actor indices: deque([])
Street index: None


In [ ]:
# Test self-play

from SelfPlayPokerGame import SelfPlayPokerGame

game = SelfPlayPokerGame()

for i in range(10):
    print(f"===============")
    print(f" New Game {i+1}")
    print("===============")
    game.run_game(verbose=True, training=False)

In [ ]:
# Test training

from SelfPlayPokerGame import SelfPlayPokerGame
import os

game = SelfPlayPokerGame()

save_path = "data/models/test"
os.makedirs(save_path, exist_ok=True)

training_stats = game.run_training_session(num_games=10, verbose=True, save_interval=5, save_path=save_path)

========= New game 1 =========
======= New Round =======
Player 0 cards: [2h, 9c], chips: 9900
Player 1 cards: [Ah, 9d], chips: 9950.0
Street index: 0
Public cards: []
Bets: P0: 100, P1: 50.0, Chips: P0: 9900, P1: 9950.0
Player 1 goes all-in for 10000.0
Street index: 0
Public cards: []
Bets: P0: 100, P1: 10000.0, Chips: P0: 9900, P1: 0.0
Player 0 folds
======= New Round =======
Player 0 cards: [3s, 3c], chips: 9800
Player 1 cards: [Td, Jh], chips: 10050.0
Street index: 0
Public cards: []
Bets: P0: 100, P1: 50.0, Chips: P0: 9800, P1: 10050.0
Player 1 bets/raises 100 to 200
Street index: 0
Public cards: []
Bets: P0: 100, P1: 200, Chips: P0: 9800, P1: 9900.0
Player 0 goes all-in for 9900
Street index: 0
Public cards: []
Bets: P0: 9900, P1: 200, Chips: P0: 0, P1: 9900.0
Player 1 matches other player's all-in
======= New Round =======
Player 0 cards: [5s, 4c], chips: 19700
Player 1 cards: [Jh, 8c], chips: 150.0
Street index: 0
Public cards: []
Bets: P0: 100, P1: 50.0, Chips: P0: 19700, P1: 

{'player0_wins': 4,
 'player1_wins': 6,
 'total_losses': [11709195.0,
  48019904.0,
  669575424.0,
  163115664.0,
  158724512.0,
  163660720.0,
  156530736.0,
  153461168.0,
  366722080.0,
  49004904.0,
  10836701.0,
  48019904.0,
  11056730.0,
  49203104.0,
  49004904.0,
  21595412.0,
  10785976.0,
  9865216.0,
  9865216.0,
  48019904.0,
  49005032.0,
  11084726.0,
  10023026.0,
  48412704.0,
  40455012.0,
  11084726.0,
  10568481.0,
  10836699.0,
  371112352.0,
  169911248.0,
  174751520.0],
 'policy_losses': [0.9315354228019714,
  0.20875409245491028,
  4.339123915997334e-05,
  -0.0,
  -0.0,
  0.06441261619329453,
  0.27577534317970276,
  -0.0,
  0.665310800075531,
  -0.0,
  1.1920928244535389e-07,
  0.007223915308713913,
  -0.0,
  -0.0,
  8.344646857949556e-07,
  0.5322357416152954,
  0.10463449358940125,
  -0.0,
  -0.0,
  -0.0,
  128.34730529785156,
  -0.0,
  0.07832145690917969,
  -0.0,
  -0.0,
  -0.0,
  -0.0,
  -0.0,
  -0.0,
  -0.0,
  -0.0],
 'value_losses': [23418388.0,
  96039

In [ ]:
from utilities import Utilities

Utilities.plot_all_stats(training_stats, save_dir=save_path)

AttributeError: 'SelfPlayPokerGame' object has no attribute 'training_stats'